# <center>Working with PDFs in Python </center>
<p><p>
<center>Mike Driscoll (@driscollis)</center>

# The Portable Document Format



# Manipulating PDFs with Python

* How to Extract Document Information From a PDF in Python
* How to Rotate Pages
* How to Merge PDFs
* How to Split PDFs
* How to Add Watermarks
* How to Encrypt a PDF

# PDF Libraries for Python

There are too many!

* **PyPDF2**: rotating, merging, splitting, watermarking, etc
* **pdfrw**: rotating, merging, splitting, watermarking, etc
* **PDFMiner / Slate**: For extracting text
* **ReportLab**: For creating PDFs
* and several others

# We Will Be Using PyPDF2

PyPDF2 may be old, but it's reliable

**pdfrw** is a viable alternative

# Versions of PyPDF

* PyPDF (dead)
* PyPDF2
* PyPDF3 (dead)
* PyPDF4 (beta)